In [2]:
using PCT, Profile

In [3]:
h, _ = @pct pullback((z::C) -> z' * z)
h

𝒫((z) -> 
    (z⋅z'))

In [4]:
println(verbose(h))

Pullback((z::C->
    (*
        z::C,
        z::C'
    )::C
::[C->C])::[C×C->C]


In [5]:
reduce_pullback(h)[1]

(z, _K) -> 
    ((z, _K) -> 
        (((z, _K) -> 
            _K)(z, (_K⋅z))+((z, _K) -> 
            ((z, _K) -> 
                _K)(z, _K'))(z, (_K⋅z'))))(z, _K)

In [6]:
p = eval_all(reduce_pullback(h)[1])
q = eval_all(call(p, first(ff(p)), constant(1)))

(z+z)

In [8]:
simplify(q) |> first

  0.000018 seconds (4 allocations: 192 bytes)
1 gcd
(z+z)
(z⋅(1+1))

  0.000026 seconds (9 allocations: 352 bytes)
2 add_const
(z⋅(1+1))
(z⋅2)
directed!

  0.000000 seconds


(z⋅2)

In [2]:
f, ctx = @pct begin
    @space H begin
        type=(I, I)->C
        symmetries=(((2,1), :conj),)
    end
    
    @space V begin
        type=(I,)->C
    end

    @space T begin
        type=(I,I,I,I) -> C
        symmetries=(((2,1,4,3), :conj), ((3,4,1,2), :id))
    end
                
    @space U begin
        type=(I,I)->C
    end
                    
    (A::H, J::T) -> _
end

(Map(MapType(VecType(AbstractPCTType[MapType(VecType(AbstractPCTType[I(), I()]), C(), Dict{Symbol, Any}(:name => :H, :symmetries => (((2, 1), :conj),))), MapType(VecType(AbstractPCTType[I(), I(), I(), I()]), C(), Dict{Symbol, Any}(:name => :T, :symmetries => (((2, 1, 4, 3), :conj), ((3, 4, 1, 2), :id))))]), UndeterminedPCTType(), Dict{Any, Any}()), PCTVector(VecType(AbstractPCTType[MapType(VecType(AbstractPCTType[I(), I()]), C(), Dict{Symbol, Any}(:name => :H, :symmetries => (((2, 1), :conj),))), MapType(VecType(AbstractPCTType[I(), I(), I(), I()]), C(), Dict{Symbol, Any}(:name => :T, :symmetries => (((2, 1, 4, 3), :conj), ((3, 4, 1, 2), :id))))]), Var{MapType}[Var{MapType}(MapType(VecType(AbstractPCTType[I(), I()]), C(), Dict{Symbol, Any}(:name => :H, :symmetries => (((2, 1), :conj),))), :A), Var{MapType}(MapType(VecType(AbstractPCTType[I(), I(), I(), I()]), C(), Dict{Symbol, Any}(:name => :T, :symmetries => (((2, 1, 4, 3), :conj), ((3, 4, 1, 2), :id)))), :J)]), Var{UndeterminedPCTTyp

In [11]:
f

(A, J) -> 
    _

In [39]:
g = @pct f ctx pullback((x::V) -> sum((i, j), x(i)' * A(i, j) * x(j)))

(A, J) -> 
    𝒫((x) -> 
        ∑((i_0, i_1), (x(i_0)⋅A(i_1, i_0)⋅x(i_1)')))

In [40]:
println(verbose(g))

(A::[I×I->C], J::[I×I×I×I->C]->
    Pullback((x::[I->C]->
        ∑((i_0::I, i_1::I),
            (*
                x::[I->C](i_0::I)::C,
                A::[I×I->C](i_1::I, i_0::I)::C,
                x::[I->C](i_1::I)::C'
            )::C
        )::C
    ::[[I->C]->C])::[[I->C]×C->[I->C]]
::[[I×I->C]×[I×I×I×I->C]->[[I->C]×C->[I->C]]]


In [41]:
p = fc(eval_all(reduce_pullback(g)))

(x, _K) -> 
    (i_2) -> 
        ∑((i_0, i_1), (δ(i_2, i_1, (x(i_0)⋅A(i_1, i_0)⋅_K'))+δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)'⋅_K))))

In [42]:
q = eval_all(call(p, first(ff(p)), constant(1)))

(i_2) -> 
    ∑((i_0, i_1), (δ(i_2, i_1, (x(i_0)⋅A(i_1, i_0)))+δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)'))))

In [43]:
simplify(q) |> first

  0.000019 seconds (7 allocations: 304 bytes)
1 sum_dist
∑((i_0, i_1), (δ(i_2, i_1, (x(i_0)⋅A(i_1, i_0)))+δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)'))))
(∑((i_0, i_1), δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)')))+∑((i_0, i_1), δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)'))))

  0.000281 seconds (293 allocations: 13.062 KiB)
2 contract_delta
(∑((i_0, i_1), δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)')))+∑((i_0, i_1), δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)'))))
(∑((i_0), (x(i_0)⋅A(i_2, i_0)))+∑((i_0, i_1), δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)'))))
directed!

  0.000041 seconds (215 allocations: 9.688 KiB)
3 contract_delta
(∑((i_0), (x(i_0)⋅A(i_2, i_0)))+∑((i_0, i_1), δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)'))))
(∑((i_0), (x(i_0)⋅A(i_2, i_0)))+∑((i_0), (x(i_0)⋅A(i_2, i_0))))
directed!

  0.000019 seconds (37 allocations: 1.625 KiB)
4 gcd
(∑((i_0), (x(i_0)⋅A(i_2, i_0)))+∑((i_0), (x(i_0)⋅A(i_2, i_0))))
((1+1)⋅∑((i_0), (x(i_0)⋅A(i_2, i_0))))

  0.000042 seconds (244 allocations: 11.312 KiB)
5 add_const
((1+1)⋅∑((i_0), (x(i_0)⋅A(i_2, i_0))))
(∑((i_0), (x

(i_2) -> 
    (∑((i_0), (x(i_0)⋅A(i_2, i_0)))⋅2)

In [44]:
g = @pct f ctx pullback((C::U) -> sum((i,j,p,q,r,s), C(p,i)' * C(q,i) * C(r,j)' * C(s,j) * J(p,q,r,s)))

(A, J) -> 
    𝒫((C) -> 
        ∑((i_0, i_1, i_2, i_3, i_4, i_5), (C(i_1, i_5)⋅C(i_3, i_4)⋅J(i_1, i_0, i_3, i_2)'⋅C(i_2, i_4)'⋅C(i_0, i_5)')))

In [45]:
p = fc(eval_all(reduce_pullback(g)))

(C, _K) -> 
    (i_6, i_7) -> 
        ∑((i_0, i_1, i_2, i_3, i_4, i_5), (δ(i_7, i_5, δ(i_6, i_2, (J(i_1, i_4, i_2, i_3)⋅C(i_4, i_0)⋅C(i_3, i_5)⋅C(i_1, i_0)'⋅_K)))+δ(i_7, i_5, δ(i_6, i_3, (C(i_1, i_0)⋅C(i_2, i_5)⋅J(i_1, i_4, i_2, i_3)'⋅C(i_4, i_0)'⋅_K')))+δ(i_7, i_0, δ(i_6, i_1, (J(i_1, i_4, i_2, i_3)⋅C(i_4, i_0)⋅C(i_3, i_5)⋅C(i_2, i_5)'⋅_K)))+δ(i_7, i_0, δ(i_6, i_4, (C(i_1, i_0)⋅C(i_2, i_5)⋅J(i_1, i_4, i_2, i_3)'⋅C(i_3, i_5)'⋅_K')))))

In [46]:
q = eval_all(call(p, first(ff(p)), constant(1)))

(i_6, i_7) -> 
    ∑((i_0, i_1, i_2, i_3, i_4, i_5), (δ(i_7, i_5, δ(i_6, i_2, (J(i_1, i_4, i_2, i_3)⋅C(i_4, i_0)⋅C(i_3, i_5)⋅C(i_1, i_0)')))+δ(i_7, i_5, δ(i_6, i_3, (C(i_1, i_0)⋅C(i_2, i_5)⋅J(i_1, i_4, i_2, i_3)'⋅C(i_4, i_0)')))+δ(i_7, i_0, δ(i_6, i_1, (J(i_1, i_4, i_2, i_3)⋅C(i_4, i_0)⋅C(i_3, i_5)⋅C(i_2, i_5)')))+δ(i_7, i_0, δ(i_6, i_4, (C(i_1, i_0)⋅C(i_2, i_5)⋅J(i_1, i_4, i_2, i_3)'⋅C(i_3, i_5)')))))

In [47]:
simplify(q) |> first

  0.000057 seconds (336 allocations: 15.594 KiB)
1 delta_ex
∑((i_0, i_1, i_2, i_3, i_4, i_5), (δ(i_7, i_5, δ(i_6, i_2, (J(i_1, i_4, i_2, i_3)⋅C(i_4, i_0)⋅C(i_3, i_5)⋅C(i_1, i_0)')))+δ(i_7, i_5, δ(i_6, i_3, (C(i_1, i_0)⋅C(i_2, i_5)⋅J(i_1, i_4, i_2, i_3)'⋅C(i_4, i_0)')))+δ(i_7, i_0, δ(i_6, i_1, (J(i_1, i_4, i_2, i_3)⋅C(i_4, i_0)⋅C(i_3, i_5)⋅C(i_2, i_5)')))+δ(i_7, i_0, δ(i_6, i_4, (C(i_1, i_0)⋅C(i_2, i_5)⋅J(i_1, i_4, i_2, i_3)'⋅C(i_3, i_5)')))))
∑((i_0, i_1, i_2, i_3, i_4, i_5), (δ(i_6, i_0, δ(i_7, i_1, (C(i_4, i_1)⋅C(i_3, i_5)⋅J(i_4, i_0, i_3, i_2)'⋅C(i_2, i_5)')))+δ(i_7, i_1, δ(i_6, i_4, (J(i_4, i_0, i_3, i_2)⋅C(i_2, i_5)⋅C(i_0, i_1)⋅C(i_3, i_5)')))+δ(i_7, i_5, δ(i_6, i_2, (C(i_4, i_1)⋅C(i_3, i_5)⋅J(i_4, i_0, i_3, i_2)'⋅C(i_0, i_1)')))+δ(i_7, i_5, δ(i_6, i_3, (J(i_4, i_0, i_3, i_2)⋅C(i_2, i_5)⋅C(i_0, i_1)⋅C(i_4, i_1)')))))

  0.000068 seconds (573 allocations: 26.312 KiB)
2 delta_ex
∑((i_0, i_1, i_2, i_3, i_4, i_5), (δ(i_6, i_0, δ(i_7, i_1, (C(i_4, i_1)⋅C(i_3, i_5)⋅J(i_4, i_0, i_3, i_2)

  0.000238 seconds (2.96 k allocations: 113.609 KiB)
11 delta_ex
∑((i_0, i_1, i_2, i_3, i_4, i_5), (δ(i_6, i_4, δ(i_7, i_1, (J(i_4, i_0, i_3, i_2)⋅C(i_2, i_5)⋅C(i_0, i_1)⋅C(i_3, i_5)')))+δ(i_6, i_0, δ(i_7, i_1, (C(i_4, i_1)⋅C(i_3, i_5)⋅J(i_4, i_0, i_3, i_2)'⋅C(i_2, i_5)')))+δ(i_7, i_5, δ(i_6, i_2, (C(i_4, i_1)⋅C(i_3, i_5)⋅J(i_4, i_0, i_3, i_2)'⋅C(i_0, i_1)')))+δ(i_7, i_5, δ(i_6, i_3, (J(i_4, i_0, i_3, i_2)⋅C(i_2, i_5)⋅C(i_0, i_1)⋅C(i_4, i_1)')))))
∑((i_0, i_1, i_2, i_3, i_4, i_5), (δ(i_6, i_0, δ(i_7, i_1, (C(i_4, i_5)⋅C(i_2, i_1)⋅J(i_4, i_3, i_2, i_0)'⋅C(i_3, i_5)')))+δ(i_7, i_1, δ(i_6, i_2, (J(i_4, i_3, i_2, i_0)⋅C(i_3, i_5)⋅C(i_0, i_1)⋅C(i_4, i_5)')))+δ(i_7, i_5, δ(i_6, i_4, (J(i_4, i_3, i_2, i_0)⋅C(i_3, i_5)⋅C(i_0, i_1)⋅C(i_2, i_1)')))+δ(i_7, i_5, δ(i_6, i_3, (C(i_4, i_5)⋅C(i_2, i_1)⋅J(i_4, i_3, i_2, i_0)'⋅C(i_0, i_1)')))))

  0.000211 seconds (2.57 k allocations: 98.469 KiB)
12 delta_ex
∑((i_0, i_1, i_2, i_3, i_4, i_5), (δ(i_6, i_0, δ(i_7, i_1, (C(i_4, i_5)⋅C(i_2, i_1)⋅J(i_4, i_3, 

  0.000442 seconds (5.49 k allocations: 229.625 KiB)
23 delta_ex
(∑((i_0, i_1, i_2, i_3, i_4, i_5), (δ(i_7, i_5, δ(i_6, i_3, (J(i_1, i_2, i_3, i_4)⋅C(i_4, i_5)⋅C(i_2, i_0)⋅C(i_1, i_0)')))+δ(i_7, i_0, δ(i_6, i_1, (J(i_1, i_2, i_3, i_4)⋅C(i_4, i_5)⋅C(i_2, i_0)⋅C(i_3, i_5)')))+δ(i_7, i_0, δ(i_6, i_2, (C(i_1, i_0)⋅C(i_3, i_5)⋅J(i_1, i_2, i_3, i_4)'⋅C(i_4, i_5)')))))+∑((i_0, i_1, i_2, i_3, i_4, i_5), δ(i_6, i_0, δ(i_7, i_1, (C(i_4, i_5)⋅C(i_2, i_1)⋅J(i_4, i_3, i_2, i_0)'⋅C(i_3, i_5)')))))
(∑((i_0, i_1, i_2, i_3, i_4, i_5), (δ(i_6, i_0, δ(i_7, i_1, (C(i_4, i_1)⋅C(i_3, i_5)⋅J(i_4, i_0, i_3, i_2)'⋅C(i_2, i_5)')))+δ(i_7, i_5, δ(i_6, i_2, (C(i_4, i_1)⋅C(i_3, i_5)⋅J(i_4, i_0, i_3, i_2)'⋅C(i_0, i_1)')))+δ(i_7, i_5, δ(i_6, i_3, (J(i_4, i_0, i_3, i_2)⋅C(i_2, i_5)⋅C(i_0, i_1)⋅C(i_4, i_1)')))))+∑((i_0, i_1, i_2, i_3, i_4, i_5), δ(i_6, i_0, δ(i_7, i_1, (C(i_4, i_5)⋅C(i_2, i_1)⋅J(i_4, i_3, i_2, i_0)'⋅C(i_3, i_5)')))))

  0.000392 seconds (5.01 k allocations: 208.125 KiB)
24 delta_ex
(∑((i_0, i_1, i_2, i

  0.000538 seconds (6.67 k allocations: 270.516 KiB)
32 delta_ex
(∑((i_0, i_1, i_2, i_3, i_4, i_5), (δ(i_6, i_0, δ(i_7, i_1, (C(i_4, i_1)⋅C(i_2, i_3)⋅J(i_4, i_0, i_2, i_5)'⋅C(i_5, i_3)')))+δ(i_7, i_1, δ(i_6, i_4, (J(i_4, i_0, i_2, i_5)⋅C(i_5, i_3)⋅C(i_0, i_1)⋅C(i_2, i_3)')))+δ(i_7, i_3, δ(i_6, i_5, (C(i_4, i_1)⋅C(i_2, i_3)⋅J(i_4, i_0, i_2, i_5)'⋅C(i_0, i_1)')))))+∑((i_0, i_1, i_2, i_3, i_4, i_5), δ(i_6, i_0, δ(i_7, i_1, (C(i_4, i_5)⋅C(i_2, i_1)⋅J(i_4, i_3, i_2, i_0)'⋅C(i_3, i_5)')))))
(∑((i_0, i_1, i_2, i_3, i_4, i_5), (δ(i_6, i_2, δ(i_7, i_1, (J(i_4, i_3, i_2, i_0)⋅C(i_3, i_5)⋅C(i_0, i_1)⋅C(i_4, i_5)')))+δ(i_6, i_0, δ(i_7, i_1, (C(i_4, i_5)⋅C(i_2, i_1)⋅J(i_4, i_3, i_2, i_0)'⋅C(i_3, i_5)')))+δ(i_7, i_5, δ(i_6, i_4, (J(i_4, i_3, i_2, i_0)⋅C(i_3, i_5)⋅C(i_0, i_1)⋅C(i_2, i_1)')))))+∑((i_0, i_1, i_2, i_3, i_4, i_5), δ(i_6, i_0, δ(i_7, i_1, (C(i_4, i_5)⋅C(i_2, i_1)⋅J(i_4, i_3, i_2, i_0)'⋅C(i_3, i_5)')))))

  0.000451 seconds (5.57 k allocations: 234.734 KiB)
33 delta_ex
(∑((i_0, i_1, i_2, i

  0.000784 seconds (11.16 k allocations: 473.625 KiB)
41 delta_ex
(∑((i_0, i_1, i_2, i_3, i_4, i_5), δ(i_6, i_0, δ(i_7, i_1, (C(i_4, i_1)⋅C(i_2, i_3)⋅J(i_4, i_0, i_2, i_5)'⋅C(i_5, i_3)'))))+∑((i_0, i_1, i_2, i_3, i_4, i_5), δ(i_6, i_0, δ(i_7, i_1, (C(i_4, i_5)⋅C(i_2, i_1)⋅J(i_4, i_3, i_2, i_0)'⋅C(i_3, i_5)'))))+∑((i_0, i_1, i_2, i_3, i_4, i_5), δ(i_6, i_0, δ(i_7, i_1, (C(i_2, i_1)⋅C(i_3, i_4)⋅J(i_2, i_0, i_3, i_5)'⋅C(i_5, i_4)'))))+∑((i_0, i_1, i_2, i_3, i_4, i_5), δ(i_7, i_0, δ(i_6, i_1, (J(i_1, i_4, i_2, i_3)⋅C(i_4, i_0)⋅C(i_3, i_5)⋅C(i_2, i_5)')))))
(∑((i_0, i_1, i_2, i_3, i_4, i_5), δ(i_6, i_0, δ(i_7, i_1, (C(i_4, i_5)⋅C(i_2, i_1)⋅J(i_4, i_3, i_2, i_0)'⋅C(i_3, i_5)'))))+∑((i_0, i_1, i_2, i_3, i_4, i_5), δ(i_6, i_0, δ(i_7, i_1, (C(i_2, i_1)⋅C(i_3, i_4)⋅J(i_2, i_0, i_3, i_5)'⋅C(i_5, i_4)'))))+∑((i_0, i_1, i_2, i_3, i_4, i_5), δ(i_7, i_0, δ(i_6, i_1, (J(i_1, i_4, i_2, i_3)⋅C(i_4, i_0)⋅C(i_3, i_5)⋅C(i_2, i_5)'))))+∑((i_0, i_1, i_2, i_3, i_4, i_5), δ(i_7, i_0, δ(i_6, i_1, (J(i_1, i_2, i

  0.000352 seconds (5.61 k allocations: 257.953 KiB)
50 contract_delta
(∑((i_0, i_1, i_2, i_3), (J(i_6, i_0, i_1, i_2)⋅C(i_2, i_3)⋅C(i_0, i_7)⋅C(i_1, i_3)'))+∑((i_0, i_1, i_2, i_3, i_4), δ(i_7, i_0, (J(i_6, i_1, i_2, i_3)⋅C(i_1, i_0)⋅C(i_3, i_4)⋅C(i_2, i_4)')))+∑((i_0, i_1, i_2, i_3, i_4, i_5), δ(i_7, i_0, δ(i_6, i_1, (J(i_1, i_2, i_5, i_3)⋅C(i_2, i_0)⋅C(i_3, i_4)⋅C(i_5, i_4)'))))+∑((i_0, i_1, i_2, i_3, i_4, i_5), δ(i_7, i_0, δ(i_6, i_1, (J(i_1, i_2, i_5, i_3)⋅C(i_2, i_0)⋅C(i_3, i_4)⋅C(i_5, i_4)')))))
(∑((i_0, i_1, i_2, i_3), (J(i_6, i_0, i_1, i_2)⋅C(i_2, i_3)⋅C(i_0, i_7)⋅C(i_1, i_3)'))+∑((i_0, i_1, i_2, i_3), (J(i_6, i_0, i_1, i_2)⋅C(i_2, i_3)⋅C(i_0, i_7)⋅C(i_1, i_3)'))+∑((i_0, i_1, i_2, i_3, i_4, i_5), δ(i_7, i_0, δ(i_6, i_1, (J(i_1, i_2, i_5, i_3)⋅C(i_2, i_0)⋅C(i_3, i_4)⋅C(i_5, i_4)'))))+∑((i_0, i_1, i_2, i_3, i_4, i_5), δ(i_7, i_0, δ(i_6, i_1, (J(i_1, i_2, i_5, i_3)⋅C(i_2, i_0)⋅C(i_3, i_4)⋅C(i_5, i_4)')))))
directed!

  0.000357 seconds (5.59 k allocations: 249.125 KiB)
51 contract

(i_6, i_7) -> 
    (∑((i_0, i_1, i_2, i_3), (J(i_6, i_0, i_1, i_2)⋅C(i_2, i_3)⋅C(i_0, i_7)⋅C(i_1, i_3)'))⋅4)

In [8]:
Profile.clear()
@profile p==p
pprof()

"profile.pb.gz"

Main binary filename not available.
Serving web UI on http://localhost:57599


In [ ]:
(A::H) -> (x::V) -> x A x